# Rheostat with VirE2
In this notebook, I will outline a possible model that used in conjunction with Agrima VirE2 Model.<br>
7.12.2020<br>
Ankita Roychoudhury

In [11]:
# bioscrape
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pandas as pd

from scipy.integrate import simps
from numpy import trapz

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
bokeh.io.output_notebook()

# from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
# from biocrnpyler import Mixture
# from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
from biocrnpyler import *

Loading BokehJS ...

## Try to combine with subsbml

In [12]:
from subsbml import createNewSubsystem, createSubsystem



In [20]:
ss1 = createSubsystem('vire2_SBML_model_hrs.xml', subsystemName = 'vire2')
ss2 = createSubsystem('ATP_rheostat.xml', subsystemName = 'rheostat')

# rename to have same things
ss1.renameSName('ATP', 'metabolite_atp')
ss1.renameSName('ADP', 'metabolite_adp')
ss1.renameSName('P', 'metabolite_pi')

<SBMLDocument>

In [21]:
atp_regen_subsystem = ss1 + ss2

In [22]:
ATP_id = atp_regen_subsystem.getSpeciesByName('metabolite_atp').getId()
VirE2_b_id = atp_regen_subsystem.getSpeciesByName('VirE2_b').getId()
VirE2_f_id = atp_regen_subsystem.getSpeciesByName('VirE2_f').getId()
atc_i_id = atp_regen_subsystem.getSpeciesByName('aTc_i').getId()




# SET RHEOSTAT INITIAL CONDITIONS
rheostat_30 = ['molecule_glucose', 'metabolite_atp', 'metabolite_nadp', 'metabolite_pi']
for spec in rheostat_30:
    atp_regen_subsystem.setSpeciesAmount(spec,30)
    if spec == 'metabolite_atp':
        atp_regen_subsystem.setSpeciesAmount(spec,10)


rheostat_e = ['enzyme_hex', 'enzyme_pgi', 'enzyme_pfk', 'enzyme_ald_tpi', 'enzyme_gapN',
             'enzyme_mGapDH', 'enzyme_pgk', 'enzyme_pgm', 'enzyme_eno', 'enzyme_pyk',
             'enzyme_alsS', 'enzyme_IlvC', 'enzyme_kivD', 'enzyme_yahk']

for spec_e in rheostat_e:
    atp_regen_subsystem.setSpeciesAmount(spec_e, 0.20)

# SET VIRE2 INITIAL CONDITIONS
atp_regen_subsystem.setSpeciesAmount('aTc_i', 15)
atp_regen_subsystem.setSpeciesAmount('dna_g', 1)
atp_regen_subsystem.setSpeciesAmount('tetR', 10)
#atp_regen_subsystem.setSpeciesAmount('ati', 15)







_ = atp_regen_subsystem.writeSBML('atp_regen_ss.xml')



In [23]:
# Simulate with Bioscrape and plot the result

#colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a']


timepoints = np.linspace(0,75,100)
results, _ = atp_regen_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)


p1 = bokeh.plotting.figure(width = 600, height = 350,
                          title = 'VirE2 with Rheostat',
                          y_axis_label = 'Concentration (mM)',
                          x_axis_label = 'Time (hrs)')

p1.line(timepoints,results[VirE2_b_id], color = colors[0], line_width = 3, legend_label = 'VirE2_b' )
p1.line(timepoints,results[atc_i_id], color = colors[1], line_width = 3, legend_label = 'atc_i' )


p1.line(timepoints,results[VirE2_f_id], color = colors[2], line_width = 3, legend_label = 'VirE2_f' )
p1.line(timepoints,results[ATP_id], color = colors[3], line_width = 3, legend_label = 'atp' )


bokeh.io.show(p1)



# plt.figure(figsize=(12,6))


# plt.plot(timepoints, results[ATP_id], linewidth = 3, label = 'ATP',  )
# plt.plot(timepoints, results[VirE2_b_id], color = 'red', lw = 4, label='VirE2_b')
# plt.plot(timepoints, results[atc_i_id], color = 'green', lw = 4, label='atc_i')
# plt.plot(timepoints, results[VirE2_f_id], color = 'cyan', lw = 2, label='VirE2_f')


# plt.title('VirE2 with ATP Rheostat')
# plt.ylabel('[X]')
# plt.xlabel('Time')
# plt.legend()
# plt.show()



In [ ]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,biocrnpyler,bioscrape